In [1]:
using POMDPs
using QuickPOMDPs
using POMDPTools: Deterministic, Uniform, SparseCat, stepthrough, RandomPolicy, FunctionPolicy

In [3]:
tiger = QuickPOMDP(
    states = ["left", "right"],
    actions = ["left", "right", "listen"],
    observations = ["left", "right"],
    
    transition = function (s, a)
        if a == "listen"
            return Deterministic(s)
        else
            return Uniform(["left", "right"])
        end
    end,
    
    observation = function (a, sp)
        if a == "listen"
            if sp == "left"
                return SparseCat(["left", "right"], [0.85, 0.15])
            else
                return SparseCat(["right", "left"], [0.85, 0.15])
            end
        else
            return Uniform(["left", "right"])
        end
    end,
    
    reward = function (s, a)
        if a == "listen"
            return -1.0
        elseif a == s
            return -100.0
        else
            return 10.0
        end
    end,
    
    initialstate = Uniform(["left", "right"]),
    
    discount = 0.95
)

QuickPOMDP{UUID("4ebb782b-890d-42cc-ae9a-fb13b0ab767a"), String, String, String, @NamedTuple{stateindex::Dict{String, Int64}, isterminal::Bool, obsindex::Dict{String, Int64}, states::Vector{String}, observations::Vector{String}, discount::Float64, actions::Vector{String}, observation::var"#16#19", actionindex::Dict{String, Int64}, transition::var"#15#18", reward::var"#17#20", initialstate::Uniform{Set{String}}}}((stateindex = Dict("left" => 1, "right" => 2), isterminal = false, obsindex = Dict("left" => 1, "right" => 2), states = ["left", "right"], observations = ["left", "right"], discount = 0.95, actions = ["left", "right", "listen"], observation = var"#16#19"(), actionindex = Dict("left" => 1, "right" => 2, "listen" => 3), transition = var"#15#18"(), reward = var"#17#20"(), initialstate = Uniform{Set{String}}(Set(["left", "right"]))))

In [4]:
for step in stepthrough(tiger, RandomPolicy(tiger), "s,a,r,sp,o", max_steps=10)
    display(step)
end

(s = "right", a = "right", r = -100.0, sp = "right", o = "right")

(s = "right", a = "listen", r = -1.0, sp = "right", o = "right")

(s = "right", a = "right", r = -100.0, sp = "right", o = "right")

(s = "right", a = "left", r = 10.0, sp = "left", o = "right")

(s = "left", a = "right", r = 10.0, sp = "right", o = "left")

(s = "right", a = "right", r = -100.0, sp = "left", o = "right")

(s = "left", a = "left", r = -100.0, sp = "right", o = "left")

(s = "right", a = "listen", r = -1.0, sp = "right", o = "right")

(s = "right", a = "left", r = 10.0, sp = "right", o = "right")

(s = "right", a = "listen", r = -1.0, sp = "right", o = "right")

In [5]:
function belief_update(m::POMDP, b, a, o)
    states = collect(support(b))
    probs = zeros(length(states))
    for i in 1:length(states)
        z = observation(m, a, states[i])
        sp = states[i]
        probs[i] = pdf(z, o)*sum(s -> pdf(b, s)*pdf(transition(m, s, a), sp), states)
    end
    probs ./= sum(probs)
    return SparseCat(states, probs)
end

belief_update (generic function with 1 method)

In [7]:
belief = Uniform(["left", "right"])
display(belief)
for step in stepthrough(tiger, FunctionPolicy(_->"left"), "s,a,r,sp,o", max_steps=10)
    display(step)
    belief = belief_update(tiger, belief, step.a, step.o)
    display(belief)
end

                      Uniform distribution            
           ┌                                        ┐ 
    "left" ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.5   
   "right" ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.5   
           └                                        ┘ 

(s = "right", a = "left", r = 10.0, sp = "right", o = "right")

                     SparseCat distribution           
           ┌                                        ┐ 
    "left" ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.5   
   "right" ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.5   
           └                                        ┘ 

(s = "right", a = "left", r = 10.0, sp = "left", o = "left")

                     SparseCat distribution           
           ┌                                        ┐ 
    "left" ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.5   
   "right" ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.5   
           └                                        ┘ 

...

                     SparseCat distribution           
           ┌                                        ┐ 
    "left" ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.5   
   "right" ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.5   
           └                                        ┘ 

(s = "right", a = "left", r = 10.0, sp = "right", o = "left")

                     SparseCat distribution           
           ┌                                        ┐ 
    "left" ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.5   
   "right" ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.5   
           └                                        ┘ 

(s = "right", a = "left", r = 10.0, sp = "left", o = "left")

                     SparseCat distribution           
           ┌                                        ┐ 
    "left" ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.5   
   "right" ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.5   
           └                                        ┘ 